In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Configurations
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 10

# Classes (alphabetical order)
classes = ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']
class_to_index = {cls: idx for idx, cls in enumerate(classes)}

# Load dataset
def load_data(root_dir):
    images = []
    labels = []
    for cls in os.listdir(root_dir):
        cls_path = os.path.join(root_dir, cls)
        if not os.path.isdir(cls_path):
            continue
        for img_name in os.listdir(cls_path):
            img_path = os.path.join(cls_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
            img = img / 255.0
            images.append(img)
            labels.append(class_to_index[cls])
    return np.array(images), np.array(labels)

root_dir = './Mushrooms'
X, y = load_data(root_dir)


In [5]:
# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()

In [6]:
train_generator = train_datagen.flow(X_train, tf.keras.utils.to_categorical(y_train, num_classes=len(classes)), batch_size=BATCH_SIZE)
val_generator = val_datagen.flow(X_val, tf.keras.utils.to_categorical(y_val, num_classes=len(classes)), batch_size=BATCH_SIZE)

# Model
base_model = MobileNetV2(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights='imagenet')
base_model.trainable = False  # Freeze backbone

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 9)              │         2,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,588,233 (9.87 MB)

 Trainable params: 330,249 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
# Train
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS, callbacks=[early_stop])

C:\Users\Dhruv\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 150s 863ms/step - accuracy: 0.2980 - loss: 2.1698 - val_accuracy: 0.5778 - val_loss: 1.2567
Epoch 2/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 135s 801ms/step - accuracy: 0.4838 - loss: 1.4566 - val_accuracy: 0.6485 - val_loss: 1.0721
Epoch 3/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 132s 788ms/step - accuracy: 0.5343 - loss: 1.3197 - val_accuracy: 0.6642 - val_loss: 1.0103
Epoch 4/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 130s 773ms/step - accuracy: 0.5672 - loss: 1.2345 - val_accuracy: 0.6821 - val_loss: 0.9416
Epoch 5/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 133s 791ms/step - accuracy: 0.5757 - loss: 1.1767 - val_accuracy: 0.6813 - val_loss: 0.9177
Epoch 6/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 128s 761ms/step - accuracy: 0.5892 - loss: 1.1528 - val_accuracy: 0.6806 - val_loss: 0.9192
Epoch 7/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 127s 756ms/step - accuracy: 0.5953 - loss: 1.1337 - val_accuracy: 0.7066 - val_loss: 0.8937
Epoch 8/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 126s 747ms/step - accuracy: 0.6177 -